In [16]:
# pip install gcloud --user

In [17]:
# pip install --upgrade google-api-python-client

In [18]:
# pip install google

In [1]:
#pip install gmaps

In [1]:
import pandas as pd
import numpy as np
import os
from google.cloud import bigquery
import geopy.distance

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'bcx-insights-6dfb9fabfb5b.json'

In [3]:
bigquery_client = bigquery.Client(project='bcx-insights')

In [4]:
sql = "SELECT * FROM `bcx-insights.bcx_networkhealth.bcxlocation_20190701_00`"

In [5]:
query = bigquery_client.query(sql)

In [6]:
df = query.to_dataframe()

In [7]:
df.head()

,Am_Id,Lat,Lon,G_City,G_Suburb,G_Street_Name,G_Str_Num,Loc_Region,Fibre_Status,Max_Cbr_Speed,Me_Status,Me_Classification,Mt_Lte
0,43337015,-25.763962,27.279501,REX,WATERKLOOF 305JQ,WATERKLOOF RD,1,NER,(blank),1000,BUILD,Intra-city,(blank)
1,44486174,-26.796704,28.93696,VAL,VAL,SMITH ST,6,NER,(blank),(blank),N,(blank),(blank)
2,43472504,-33.542736,25.665624,ADDO,ADDO,ADDO RD,1,SR,(blank),(blank),WORKING,Inter-city,(blank)
3,43472284,-33.551408,25.68969,ADDO,ADDO,R335 RD,113254,SR,(blank),(blank),WORKING,Inter-city,(blank)
4,43472289,-33.551408,25.68969,ADDO,ADDO,R335 RD,113296,SR,(blank),(blank),WORKING,Inter-city,(blank)


In [8]:
df.tail()

,Am_Id,Lat,Lon,G_City,G_Suburb,G_Street_Name,G_Str_Num,Loc_Region,Fibre_Status,Max_Cbr_Speed,Me_Status,Me_Classification,Mt_Lte
24011,49109521,-26.764218,26.420475,HARTBEESFONTEIN (NORTH WEST),HARTBEESFONTEIN,EENHEID VOORTRKKER RD,61,NER,(blank),(blank),WORKING,Inter-city,(blank)
24012,49108597,-25.103824,30.475046,HARTBEESFONTEIN (NORTH WEST),HARTBEESFONTEIN,DU TOIT ST,1,NER,(blank),(blank),WORKING,Inter-city,(blank)
24013,49109111,-26.756583,26.431264,HARTBEESFONTEIN (NORTH WEST),HARTBEESFONTEIN,MEULE ST,1,NER,(blank),(blank),WORKING,Inter-city,(blank)
24014,49109535,-26.765741,26.420132,HARTBEESFONTEIN (NORTH WEST),HARTBEESFONTEIN,Z..EENHEID ST,92,NER,(blank),(blank),WORKING,Inter-city,(blank)
24015,49109379,-26.759122,26.42373,HARTBEESFONTEIN (NORTH WEST),HARTBEESFONTEIN,EENHEID VOORTRKKER RD,113,NER,(blank),(blank),WORKING,Inter-city,(blank)


In [9]:
def format_table(df):
    
    df = df.copy()[['Am_Id','Lat','Lon','G_City','G_Suburb','G_Street_Name','G_Str_Num']]
    df.replace('(blank)', np.nan, inplace=True)
    
    new_cols = ['Country','Province','Building','Floor','Room','Postal_Code']
    for i in new_cols:
        df[i] = np.nan
        
    df['City'] = df['G_City']
    df['Suburb'] = df['G_Suburb']
    df['Street'] = df['G_Street_Name']
    df['Number'] = df['G_Str_Num']
    df['Latitude'] = df['Lat']
    df['Longitude'] = df['Lon']
    
    df = df[['Am_Id','Country','Province','City','Suburb','Street','Number','Building','Floor','Room','Postal_Code','Latitude','Longitude']]
    
    #df.sort_values(['Country','Province','City','Suburb','Street'], ascending = [True, True, True, True, True], inplace=True)
    
    return df

In [10]:
format_table(df).head(10)

,Am_Id,Country,Province,City,Suburb,Street,Number,Building,Floor,Room,Postal_Code,Latitude,Longitude
0,43337015,NaN,NaN,REX,WATERKLOOF 305JQ,WATERKLOOF RD,1,NaN,NaN,NaN,NaN,-25.763962,27.279501
1,44486174,NaN,NaN,VAL,VAL,SMITH ST,6,NaN,NaN,NaN,NaN,-26.796704,28.93696
2,43472504,NaN,NaN,ADDO,ADDO,ADDO RD,1,NaN,NaN,NaN,NaN,-33.542736,25.665624
3,43472284,NaN,NaN,ADDO,ADDO,R335 RD,113254,NaN,NaN,NaN,NaN,-33.551408,25.68969
4,43472289,NaN,NaN,ADDO,ADDO,R335 RD,113296,NaN,NaN,NaN,NaN,-33.551408,25.68969
5,43472327,NaN,NaN,ADDO,ADDO,R335 RD,937,NaN,NaN,NaN,NaN,-33.551408,25.68969
6,43472503,NaN,NaN,ADDO,ADDO,SLAGBOOM FARM ST,1,NaN,NaN,NaN,NaN,-33.374949,25.668648
7,43472251,NaN,NaN,ADDO,ADDO,ADDO PATERSON MAIN RD,11359,NaN,NaN,NaN,NaN,-33.546711,25.690003
8,43472286,NaN,NaN,ADDO,ADDO,R335 RD,113271,NaN,NaN,NaN,NaN,-33.551408,25.68969
9,43472736,NaN,NaN,ADDO,VALENCIA,IVY CT,962,NaN,NaN,NaN,NaN,-33.551963,25.699362


In [11]:
df = format_table(df)

In [12]:
# SA Booundary corner coords
SA_NW = (-23.307300, 15.692738)
SA_NE = (-22.073258, 32.890010)
SA_SW = (-34.810614, 15.664865)
SA_SE = (-34.242325, 35.264604)

In [13]:
# Geocoord Ranges
lat_range = (SA_NE[0],SA_SW[0])
lon_range = (SA_NW[1],SA_NE[1])

In [14]:
SA_NE[1]

32.89001

In [15]:
def max_dist_N(df, distance=1):
    '''Returns another column with a list of site Ids that are within the specified distance from each site in the DataFrame'''
    
    # Handy variables
    IDs = df['Am_Id']
    lat = df['Latitude']
    lon = df['Longitude']
    
    # Create a dictionary to populate
    dist_dic = {}
    
    # For nested loop populating dictionary
    for i in range(len(IDs)):
        dist_dic[IDs[i]] = []
        current_site = (lat[i],lon[i])
        for j in range(len(IDs)):
            if (df['Suburb'][i] == df['Suburb'][j]):
                other_site = (lat[j], lon[j])
                
                # Conditionally append other site IDs based on distance from current site
                if (i!=j) & (geopy.distance.great_circle(current_site, other_site) < distance):
                    dist_dic[IDs[i]].append(df['Am_Id'][j])
    
    # Transform dictionary values into an added column to the Dataframe
    df['Max_Distance_N'] = [x for x in dist_dic.values()]
    
    return df

In [16]:
max_dist_N(df.copy()[:500], distance=0.5)

,Am_Id,Country,Province,City,Suburb,Street,Number,Building,Floor,Room,Postal_Code,Latitude,Longitude,Max_Distance_N
0,43337015,NaN,NaN,REX,WATERKLOOF 305JQ,WATERKLOOF RD,1,NaN,NaN,NaN,NaN,-25.763962,27.279501,[]
1,44486174,NaN,NaN,VAL,VAL,SMITH ST,6,NaN,NaN,NaN,NaN,-26.796704,28.93696,[]
2,43472504,NaN,NaN,ADDO,ADDO,ADDO RD,1,NaN,NaN,NaN,NaN,-33.542736,25.665624,[]
3,43472284,NaN,NaN,ADDO,ADDO,R335 RD,113254,NaN,NaN,NaN,NaN,-33.551408,25.68969,"[43472289, 43472327, 43472286]"
4,43472289,NaN,NaN,ADDO,ADDO,R335 RD,113296,NaN,NaN,NaN,NaN,-33.551408,25.68969,"[43472284, 43472327, 43472286]"
5,43472327,NaN,NaN,ADDO,ADDO,R335 RD,937,NaN,NaN,NaN,NaN,-33.551408,25.68969,"[43472284, 43472289, 43472286]"
6,43472503,NaN,NaN,ADDO,ADDO,SLAGBOOM FARM ST,1,NaN,NaN,NaN,NaN,-33.374949,25.668648,[]
7,43472251,NaN,NaN,ADDO,ADDO,ADDO PATERSON MAIN RD,11359,NaN,NaN,NaN,NaN,-33.546711,25.690003,[]
8,43472286,NaN,NaN,ADDO,ADDO,R335 RD,113271,NaN,NaN,NaN,NaN,-33.551408,25.68969,"[43472284, 43472289, 43472327]"
9,43472736,NaN,NaN,ADDO,VALENCIA,IVY CT,962,NaN,NaN,NaN,NaN,-33.551963,25.699362,[]


In [17]:
df = max_dist_N(df.copy()[:1000], distance=0.5)

In [20]:
def Same_Block(df):
    '''Returns another column with a list of site Ids that are within the specified distance from each site in the DataFrame'''
    
    # Handy variables
    IDs = df['Am_Id']
    lat = df['Latitude']
    lon = df['Longitude']
    neighbours = df['Max_Distance_N']
    
    # Create a dictionary to populate
    dist_dic = {}
    
    # Nested for loop populating dictionary
    for i in range(len(IDs)):
        dist_dic[IDs[i]] = []
        current_site = (lat[i],lon[i])
        for j in neighbours[i]:
            for k in range(len(df)):
                if IDs[k]==j:
                    other_site = (lat[k],lon[k])
                    # Conditionally append other site IDs based on distance from current site
                    if geopy.distance.great_circle(current_site, other_site) < 0.1:
                        dist_dic[IDs[i]].append(j)
     
    df['Same_Block_N'] = [x for x in dist_dic.values()]
                
    return df

In [21]:
Same_Block(df)

,Am_Id,Country,Province,City,Suburb,Street,Number,Building,Floor,Room,Postal_Code,Latitude,Longitude,Max_Distance_N,Same_Block_N
0,43337015,NaN,NaN,REX,WATERKLOOF 305JQ,WATERKLOOF RD,1,NaN,NaN,NaN,NaN,-25.763962,27.279501,[],[]
1,44486174,NaN,NaN,VAL,VAL,SMITH ST,6,NaN,NaN,NaN,NaN,-26.796704,28.93696,[],[]
2,43472504,NaN,NaN,ADDO,ADDO,ADDO RD,1,NaN,NaN,NaN,NaN,-33.542736,25.665624,[],[]
3,43472284,NaN,NaN,ADDO,ADDO,R335 RD,113254,NaN,NaN,NaN,NaN,-33.551408,25.68969,"[43472289, 43472327, 43472286]","[43472289, 43472327, 43472286]"
4,43472289,NaN,NaN,ADDO,ADDO,R335 RD,113296,NaN,NaN,NaN,NaN,-33.551408,25.68969,"[43472284, 43472327, 43472286]","[43472284, 43472327, 43472286]"
5,43472327,NaN,NaN,ADDO,ADDO,R335 RD,937,NaN,NaN,NaN,NaN,-33.551408,25.68969,"[43472284, 43472289, 43472286]","[43472284, 43472289, 43472286]"
6,43472503,NaN,NaN,ADDO,ADDO,SLAGBOOM FARM ST,1,NaN,NaN,NaN,NaN,-33.374949,25.668648,[],[]
7,43472251,NaN,NaN,ADDO,ADDO,ADDO PATERSON MAIN RD,11359,NaN,NaN,NaN,NaN,-33.546711,25.690003,[],[]
8,43472286,NaN,NaN,ADDO,ADDO,R335 RD,113271,NaN,NaN,NaN,NaN,-33.551408,25.68969,"[43472284, 43472289, 43472327]","[43472284, 43472289, 43472327]"
9,43472736,NaN,NaN,ADDO,VALENCIA,IVY CT,962,NaN,NaN,NaN,NaN,-33.551963,25.699362,[],[]
